In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install pyforest
from pyforest import *
import datetime, pickle, copy
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 150)
import matplotlib.pyplot as plt
%matplotlib inline  
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
!pip install quandl
import quandl
plt.style.use('ggplot')
from statistics import variance 
from random import randint
import scipy as sp
from scipy import stats
!pip install ffn
import ffn
!pip install --upgrade pip

In [ ]:
# Natural Gas continuous contract
print('\033[4mBrent Crude Futures, Continuous Contract\033[0m')
BC = quandl.get("CHRIS/ICE_B1", authtoken="LSQpgUzwJRoF667ZpzyL") # natural gas continuous contract 1
BC = BC.loc['2010-01-01':,]
BC.sort_index(ascending=True, inplace=True)
BC.tail()

In [ ]:
print(BC.columns); print(BC.shape)

In [ ]:
import seaborn as sns
%matplotlib inline
print(BC.isnull().sum())
sns.heatmap(BC.isnull(), yticklabels = False)
plt.show()

In [ ]:
BC.drop(['EFP Volume','EFS Volume','Block Volume',
        'Change', 'Wave'],axis=1, inplace=True)
print(BC.columns)
print('\n')
BC= BC.fillna(method='ffill')
print(BC.isnull().sum())

In [ ]:
BC.tail()

In [ ]:
BC.describe()

In [ ]:
import plotly.graph_objects as go
fig = go.Figure(data=go.Ohlc(x=BC.index,
                open=BC['Open'],
                high=BC['High'],
                low=BC['Low'],
                close=BC['Settle']))

fig.update_layout(
    title='Brent Crude Futures, Continuous Contract',
    yaxis_title='Price (USD)'
)
fig.show()

In [ ]:
# Calculate the daily percentage change which is daily return
data = BC["2018":].copy()

print('\033[1m' + 'Daily percentage change:' + '\033[1m')
daily_ret = data['Settle'].pct_change().dropna()
mean_return = daily_ret.mean()
return_stdev = daily_ret.std()
print('Average daily return : %1.2f%% ' % round((mean_return*100),2))
print('Average Volatility : %1.2f%% ' % round((return_stdev*100), 2))

In [ ]:
daily_ret.plot(figsize=(10,6),grid=True)
plt.title('Daily returns')
plt.show()

In [ ]:
print('\033[4mCritical Values\033[0m')
n = len(daily_ret)
test_statistic = ((daily_ret.mean() - 0) / (daily_ret.std()/np.sqrt(n)))
print ('t test statistic: ', round(test_statistic,2))
print('\n')

from scipy.stats import t
p_val = 2 * (1 - t.cdf(test_statistic, n - 1))
print ('P-value is: ', round(p_val,1))
print('\n')

from scipy.stats import chi2
# Here we calculate the critical value directly because our df is too high for most chisquare tables
crit_value = chi2.ppf(0.99, (n - 1))
print ('Critical value at α = 0.01 with 251 df: ', round(crit_value,2))
print('\n')

# Plot the distributions
fig = plt.figure(figsize=(10,5))
sns.set(rc={'figure.figsize': (15,5)})
ax1 = fig.add_axes([0.1,0.1,0.8,0.8])
daily_ret.plot.hist(bins = 60)
ax1.set_xlabel("Daily returns %")
ax1.set_ylabel("Percent")
ax1.set_title("Brent Crude Oil daily returns")
ax1.text(-0.15,30,"Extreme Low\nreturns")
ax1.text(0.10,30,"Extreme High\nreturns")
plt.show()
print('\n')
print("Skewness : ", round(daily_ret.skew(),2))
print("Kurtosis : ", round(daily_ret.kurtosis(),2))

In [ ]:
print('\033[4mProbability of +/-(1%); +/-(3%); +/-%(5) change in price (Data -> 2018- till date)\033[0m')

print ("The probability of price changes between 1%% and -1%% is %1.2f%% " % 
       (100*daily_ret[(daily_ret > -0.01) & (daily_ret < 0.01)].shape[0] / daily_ret.shape[0]))
print ("The probability of price changes between 3%% and -3%% is %1.2f%% " % 
       (100*daily_ret[(daily_ret > -0.03) & (daily_ret < 0.03)].shape[0] / daily_ret.shape[0]))
print ("The probability of price changes between 5%% and -5%% is %1.2f%% " % 
       (100*daily_ret[(daily_ret > -0.05) & (daily_ret < 0.05)].shape[0] / daily_ret.shape[0]))
print ("The probability of price changes more than 5%% is %1.2f%%" % 
       (100*daily_ret[daily_ret > 0.05].shape[0] / daily_ret.shape[0]))
print ("The probability of price changes less than -5%% is %1.2f%%" % 
       (100*daily_ret[daily_ret < -0.05].shape[0] / daily_ret.shape[0]))

In [ ]:
print('\033[4mMinimum price [2018- till date]\033[0m')
print(round(data['Settle'].min(),2), data['Settle'].idxmin());
print('\033[4mMaximum price [2018- till date]\033[0m')
print(round(data['Settle'].max(),2), data['Settle'].idxmax());
print('\n')

print('\033[4mMinimum daily % return [2018- till date]\033[0m')
print(round(daily_ret.min(),2)*100, daily_ret.idxmin()); 
print('\033[4mMaximum daily % return [2018- till date]\033[0m')
print(round(daily_ret.max()*100, 2), daily_ret.idxmax());

In [ ]:
# Get the number of days
days = (data.index[-1] - data.index[0]).days

# Calculate the CAGR 
cagr = ((((data['Settle'][-1]) / data['Settle'][1])) ** (252.0/days)) - 1

# Print the CAGR
print('CAGR:',round(cagr*100))

In [ ]:
perf = BC['Settle'].calc_stats()
print('\n')
perf.display()

In [ ]:
perf.stats

In [ ]:
qq = BC['2015':].copy()
qq = qq[['Open', 'High', 'Low', 'Settle', 'Volume']]
print(qq)

In [ ]:
qq['h_o'] = (BC['High']- BC['Open']) # distance between Highest and Opening price
qq['l_o'] = (BC['Low'] - BC['Open']) # distance between Lowest and Opening price
qq['gain'] = (BC['Settle']- BC['Open'])
qq['dailyChange'] = (qq['Settle'] - qq['Open']) / qq['Open']
qq['closeReturn'] = BC['Settle'].pct_change()
qq['priceDirection'] = (qq['Settle'].shift(-1) - qq['Settle'])


# lags = 3
# Create the shifted lag series of prior trading period close values
# for i in range(0, lags):
    # qq["Lag%s" % str(i+1)] = BC["Settle"].shift(i+1).pct_change()
    
# qq['HL'] = (BC['High'] - BC['Settle']) / BC['Settle'] 
# creating more features
qq['volIncrement'] = BC.Volume.diff() / BC.Volume
# qq['pdoi'] = BC['Prev. Day Open Interest'].pct_change()
# qq = qq.replace([np.inf, -np.inf], np.nan)
qq = qq.dropna()
qq.head()

In [ ]:
qq.tail()

In [ ]:
print(qq.shape)

In [ ]:
# positive value = 1, otherwise, 0
qq.loc[:,"target"] = np.where(qq['priceDirection']> 0, 1.0, 0.0)
qq.drop(['priceDirection'],1, inplace=True)
qq.head()

In [ ]:
qq.tail()

In [ ]:
import seaborn as sns
sns.countplot(x = 'target', data=qq, hue='target')
plt.show()

In [ ]:
import matplotlib
matplotlib.use('Agg')

X = qq.drop(columns= ['target', 'High',
                     'Open', 'Low', 'Settle',
                     'Volume','target'], axis=1)
y = qq.target.astype(np.integer) 

In [ ]:
X.tail()

In [ ]:
pip install tscv

In [ ]:
from tscv import GapKFold

# # Create training and test sets
gkcv = GapKFold(n_splits=5, gap_before=2, gap_after=1)

"""
Introduced gaps between the training and test set to mitigate the temporal dependence.
Here the split function splits the data into Kfolds. 
The test sets are untouched, while the training sets get the gaps removed
"""

for tr_index, te_index in gkcv.split(X, y):
    xTrain, xTest = X.values[tr_index], X.values[te_index];
    yTrain, yTest = y.values[tr_index], y.values[te_index];
        
print('Observations: %d' % (len(xTrain) + len(xTest)))
print('Training Observations: %d' % (len(xTrain)))
print('Testing Observations: %d' % (len(xTest)))

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import model_selection

In [ ]:
xgb = XGBClassifier()
logreg2 = LogisticRegression(solver='lbfgs')
knn = KNeighborsClassifier(5)
lda = LinearDiscriminantAnalysis()
qda = QuadraticDiscriminantAnalysis()
lsvc = LinearSVC()
rsvm = SVC(C=1000000.0, cache_size=200, class_weight=None,
                       coef0=0.0, degree=3, gamma=0.0001, kernel='rbf',
                       max_iter=-1, probability=False, random_state=None,
                       shrinking=True, tol=0.001, verbose=False)
rfc = RandomForestClassifier(
              n_estimators=1000, criterion='gini',
              max_depth=None, min_samples_split=2,
              min_samples_leaf=1, max_features='auto',
              bootstrap=True, oob_score=False, n_jobs=1,
              random_state=None, verbose=0)

# prepare configuration for cross validation test harness
seed = 42
# prepare models
models = []
models.append(('XGB', XGBClassifier()))
models.append(('LR', LogisticRegression(solver='lbfgs',penalty='l2',random_state = 0)))
models.append(('KNN', KNeighborsClassifier()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('RF', RandomForestClassifier(
              n_estimators=1000, criterion='gini',
              max_depth=None, min_samples_split=2,
              min_samples_leaf=1, max_features='auto',
              bootstrap=True, oob_score=False, n_jobs=1,
              random_state=None, verbose=0)))
models.append(('QDA', QuadraticDiscriminantAnalysis()))
models.append(('LSVC', LinearSVC()))
models.append(('RSVM', SVC(C=1000000.0, cache_size=200, class_weight=None,
                       coef0=0.0, degree=3, gamma=0.0001, kernel='rbf',
                       max_iter=-1, probability=False, random_state=None,
                       shrinking=True, tol=0.001, verbose=False)))

# evaluate each model in turn
results = []
names = []
scoring = 'accuracy'
for name, model in models:
    #kfold = model_selection.KFold(n_splits=10, random_state=seed)
    cv_results = model_selection.cross_val_score(model, xTrain, yTrain, cv=gkcv, 
                                                 scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score

LR = LogisticRegression(solver='lbfgs',penalty='l2',random_state = 0)
LDA = LinearDiscriminantAnalysis()

ensembleModel = VotingClassifier(
    estimators=[
                ('LDA', LDA), 
                ('LR', LR)
                ],
    voting='hard'
    ).fit(X, y)

print(cross_val_score(ensembleModel, X, y, cv=gkcv).mean())


In [ ]:
modelExperiment = LinearDiscriminantAnalysis().fit(X, y)
print(cross_val_score(modelExperiment, X, y, cv=gkcv).mean())


In [ ]:
# from sklearn.decomposition import PCA
# whiten = False
# random_state = 2020

# pca_2c = PCA(n_components=2, whiten = whiten, random_state = random_state)
# x_pca_2c = pca_2c.fit_transform(xTrain)
# print(x_pca_2c.shape)
# print(pca_2c.explained_variance_ratio_.sum()); print()

In [ ]:
# from mlxtend.plotting import plot_decision_regions
# rsvm = SVC(C=1000000.0, cache_size=200, class_weight=None,
                       # coef0=0.0, degree=3, gamma=0.0001, kernel='rbf',
                       # max_iter=-1, probability=False, random_state=None,
                       # shrinking=True, tol=0.001, verbose=False).fit(x_pca_2c,yTrain)
# yhat = rsvm.predict(x_pca_2c)

# plot decision region to visualize
# plot_decision_regions(x_pca_2c, yTrain, clf=rsvm, legend=2)
# axes annotations
# plt.xlabel('comp_1'); plt.ylabel('comp_2')
# plt.title('Decision boundary')
# plt.show()

In [ ]:
from sklearn.metrics import classification_report
yhat = ensembleModel.predict(X)
print('\n Classification Report:\n', classification_report(y, yhat))

In [ ]:
yhat = pd.DataFrame(yhat)
yhat.index = y.index
yhat

In [ ]:
qq['PredictedSignal'] = yhat
qq

In [ ]:
prices = qq[['Settle', 'closeReturn', 'target', 'PredictedSignal']]['2021-01-01':].copy()
prices.tail()


In [ ]:
prices['cumReturns'] = 100*prices['closeReturn'].cumsum()
prices['cumStrategyReturns']= 100*(prices['closeReturn'] * prices['PredictedSignal'].shift(1)).cumsum()

plt.figure(figsize=(15,5))
plt.plot(prices['cumReturns'], label = 'Crude Oil returns')
plt.plot(prices['cumStrategyReturns'], label = 'Strategy returns')
plt.legend(loc='best')
plt.show()

In [ ]:
print('Number of trades (buy) = ', (prices['PredictedSignal']==1).sum())
print('Number of trades (sell) = ', (prices['PredictedSignal']==0).sum())

In [ ]:
"""
we will limit the number of orders by restricting ourselves to the number of positions on the market. 
we will apply diff() to the column signal:
"""
prices['Strategy'] = prices['PredictedSignal'].diff(2)
prices

In [ ]:
print('Number of trades (buy) = ', (prices.Strategy!=1).sum())
print('Number of trades (sell) = ', (prices.Strategy!=-1).sum())

In [ ]:
print(prices.Strategy.value_counts())

# Buy/Sell signals plot
buys = prices.loc[prices["Strategy"] == 1];
sells = prices.loc[prices["Strategy"] == -1];

# Plot
fig = plt.figure(figsize=(20, 5));
plt.plot(prices.index, prices['Settle'], lw=2., label='Price');

# Plot buy and sell signals
# up arrow when we buy one share
plt.plot(sells.index, prices.loc[sells.index]['Settle'], '^', markersize=10, color='g', lw=2., label='Buy');
# down arrow when we sell one share
plt.plot(buys.index, prices.loc[buys.index]['Settle'], 'v', markersize = 10, color='r', lw=2., label='Sell');
plt.ylabel('Price (USD)'); plt.xlabel('Date');
plt.title('Buy and Sell signals'); plt.legend(loc='best');
plt.show()

In [ ]:
dq = BC.tail(620)
lows = BC['Low']
highs = BC['High']

fig = plt.figure()
ax1 = fig.add_subplot(111, ylabel='Crude Oil price in $')
highs.plot(ax=ax1, color='c', lw=2.)
lows.plot(ax=ax1, color='y', lw=2.)
plt.hlines(highs.head(200).max(),lows.index.values[0],
           lows.index.values[-1],linewidth=2, color='g')
plt.hlines(lows.head(200).min(),lows.index.values[0],
           lows.index.values[-1], linewidth=2, color='r')
plt.axvline(linewidth=2,color='b',x=lows.index.values[200],linestyle=':')
plt.show()

In [ ]:
#strategy_std = qq.cum_strategy_returns.std().np.sqrt(252)
# print('Sharpe Ratio:',qq.cum_returns.mean() / qq.cum_returns.std())

In [ ]:
prices = prices.reset_index()
prices.tail()

In [ ]:
class Portfolio:
    def __init__(self):
        self.lotA = 1
        self.lotB = 1
        self.contract = 1
        self.initialCash = 10000
        self.buy = (np.where(prices.Strategy == 1, 
                                        self.lotA * self.contract *prices.Settle, 0))
        self.sell = (np.where(prices.Strategy == (-1), 
                                        self.lotB * self.contract *prices.Settle, 0))
        
    def long_amt(self):
        self.buy = (np.where(prices.Strategy == 1,
                                     Portfolio().lotA * 
                                       Portfolio().contract *prices.Settle, 0))
        return self.buy
        
    def cashDelta(self):
        self.cashDelta = Portfolio().buy + Portfolio().sell
        return self.cashDelta
    
    def endBalance(self):
        self.endBalance = Portfolio().initialCash + Portfolio().cashDelta().cumsum()
        return self.endBalance
    
    def endPosition(self):
        self.endPosition = prices.Strategy.cumsum()
        return self.endPosition
        
p = Portfolio()
prices['Buy'] = p.buy
prices['Sell'] = p.sell
prices['cashDelta'] = p.cashDelta()
prices['endBalance'] = p.endBalance()
prices['endPosition'] = p.endPosition()
prices.loc[:, ['Date', 'Settle', 'Strategy','Buy', 'Sell', 
                 'cashDelta', 'endBalance', 'endPosition']].tail()

In [ ]:
prices['pnl'] = prices['endBalance'] + (prices.endPosition * prices.Settle * Portfolio().contract)
prices.loc[:, ['Date', 'Settle', 'endBalance', 'endPosition', 'pnl']].tail(10)

In [ ]:
d = prices.set_index('Date')
plt.figure(figsize=(15,5))
plt.plot(d.pnl)